## Agents in Teams

In this section we will learn how to build multi-agent teams.

Different Type of Teams Architecture:

#### **RoundRobinGroupChat()**
It is a simple team configuration where all the agents share the same context and take turns responding in a round robin fashion. Each agent during its turn broadcast its response to all other agents, ensuring that the entire team maintains the same context.


In [2]:
import os
from dotenv import load_dotenv
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.messages import TextMessage

load_dotenv()
open_api_key = os.getenv('OPENAI_API_KEY')
openai_model_client = OpenAIChatCompletionClient(model='gpt-4o-mini')

### Single Agent vs MultiAgent

In [ ]:
single_agent = AssistantAgent(
    name='singlestoryagent',
    model_client=openai_model_client,
    system_message='You are a creative writer. Generate a short story about the plot user provides you.'
)

async def test_single_agent():
    
    task = TextMessage(content='Write a short story about a knight and a dog. Keep it in 300 words', source='user')
    result = await single_agent.run(task=task)
    
    print(result.messages[-1].content)
    
await test_single_agent()

Once in the bustling kingdom of Eldoria, a valiant knight named Sir Cedric roamed the fields, clad in shining armor. His noble deeds were many, but none could bring him joy in solitude. One day, while traversing the edge of the Whispering Woods, a rustle in the underbrush caught his attention.

From the thicket, a scruffy dog emerged, limping and trembling. Its coat was matted, and its eyes, filled with fear, begged for comfort. Sir Cedric knelt, extending a hand. “Fear not, little friend. I won’t harm you.” The dog sniffed cautiously before inching closer, drawn by the knight’s gentle voice.

From that day on, the dog, whom Cedric named Bramble, followed him everywhere. They became inseparable companions, sharing adventures and laughter, with Bramble’s playful antics brightening the knight’s heart. Together, they faced dragons and fierce bandits, Bramble proving to be both brave and clever.

One fateful day, news swept through the kingdom; the dark sorceress, Malvina, threatened to un

### Agents Team using RoundRobinGroupChat()

In [8]:
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console

plot_agent = AssistantAgent(
    name='plot_agent',
    model_client=openai_model_client,
    system_message='You are a creative writer. You create engaging plots for stories. Focus on creating the plot for the story based on the user prompt. You will only create the plot of the story'
)

character_agent = AssistantAgent(
    name= 'character_agent',
    model_client=openai_model_client,
    system_message='You are a creative writer. You develop each character of the stories in detals, incluing their motivation and background based on the plot provided by "plot_agent". Make sure you only develop the characters and their background and motivation.'
)

editing_agent = AssistantAgent(
    name = 'editing_agent',
    model_client=openai_model_client,
    system_message='You are a creative writer. You will write the complete story based on the plot provided by "plot_agent" and characters and their backgroud provided by "character_agent"'
)

In [9]:
## agent team
agent_team = RoundRobinGroupChat(
    participants=[plot_agent,character_agent,editing_agent],
    max_turns=3
)

async def test_team_output():
    
    task = TextMessage(content='Write a short story about a knight and a dog. Keep it in 300 words', source='user')
    result = await agent_team.run(task=task)
    
    for each_message in result.messages:
        print(f'{each_message.source} : {each_message.content}')
        print('\n\n')
        
await test_team_output()

user : Write a short story about a knight and a dog. Keep it in 300 words



plot_agent : In a realm where shadows stretched long and days of glory seemed but a tale of old, Sir Cedric, a knight of renowned honor, found himself cast out from his kingdom after a betrayal unraveled his loyalty. With heavy armor and a heart burdened by the weight of his lost home, he wandered the outskirts of the enchanted Eldergrove Forest.

As twilight draped its veil over the land, Cedric spotted a flickering light through the trees. Following it with cautious steps, he stumbled upon a remote village plagued by a fierce wolf that had tormented its inhabitants. The villagers eyed him with skepticism, their desperation palpable. In that moment, he felt a tug at his chainmail – a scruffy dog with matted fur stood at his side, wide-eyed and hopeful.

Cedric named the dog Bristle, a fitting title for its bristly coat and fierce spirit. Together, they devised a plan to confront the rogue wolf that had terror